In [ ]:
from models.GA.baseChromosome import LinearChromosome, Chromosome
import copy
import torch
import numpy as np
from models.TS.GRUPipeline import GRUwithFNO
from torch.utils.data import DataLoader, TensorDataset

import boto3

In [ ]:
SYMBOL = "SOLUSDT"

session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCAJEEvXFkR_Vlz_q0TNbK1f7',
    aws_secret_access_key='YCPw07kPNEgOJVF0N93yReOErAl7RP0-5woP_Bgl',
    region_name='ru-central1'
)

bucket_name = 'test-actions'

x_train_file = f'train/train_x_{SYMBOL}.pt'
y_train_file = f'train/train_y_{SYMBOL}.pt'
x_test_file = f'test/test_x_{SYMBOL}.pt'
y_test_file = f'test/test_y_{SYMBOL}.pt'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

s3.download_file(bucket_name, x_train_file, x_train_file[6:])
s3.download_file(bucket_name, y_train_file, y_train_file[6:])
s3.download_file(bucket_name, x_test_file, x_test_file[5:])
s3.download_file(bucket_name, y_test_file, y_test_file[5:])

print(x_train_file[6:])
x_train, y_train = torch.load(f'train_x_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device), torch.load(
    f'train_y_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device)
x_test, y_test = torch.load(f'test_x_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device), torch.load(
    f'test_y_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device)

mean, std = x_train.squeeze().reshape(-1).detach().mean().item(), x_train.squeeze().reshape(
    -1).detach().std().item()
dataset = TensorDataset(x_train, y_train)
validation = TensorDataset(x_test, y_test)